## Authentication

You may be required to authenticate with your Arup credentials to use DesignCheck2.

In [47]:
import sys
sys.path.append('../../../Scripts')

from FCU_dataclasses import FCU_batchcalc, fcu_calc
import ddb_get_post as pyddb
import ipywidgets as widgets
import pandas as pd

### Enter Job Number

In [48]:
pyddb.site = "sandbox"
job_number = "11252300"
fcu_subsystem_name = "Fan Coil Units"

### Select Building

In [49]:
project_id = pyddb.get_project_id(job_number)
building_type_id = 'dbeac84d-9235-47fb-ae08-c8d47e00f253'
project_buildings = pyddb.get_assets(project_id = project_id, asset_type_id = building_type_id)
buildings_dropdown = widgets.Dropdown(options = [(x["name"], x["id"]) for x in project_buildings])
buildings_dropdown

Dropdown(options=(('B2B3', 'db628c16-8677-468e-bb7a-97645c9f48af'),), value='db628c16-8677-468e-bb7a-97645c9f4…

### Get Site, FCU Subsystem, Parameters

In [50]:
global_parameter_ids = ['ab3dc303-4013-4abe-9b9d-7debe73b1dd4',
                        '0c41f0d7-22c1-47f2-9e34-579182414027',
                        '5208e0ae-fbbe-4073-83d7-dcfed2364f01',
                        '91daecd8-663a-40ea-b6c2-43dc6170656d',
                        '4f96a046-3ba3-44de-abac-86dffe0eb77d',
                        'b7bfdea1-f073-4cb9-9ce3-5650ea5b25e6',
                        'ff68d5af-c965-47ee-9726-db386f0cc11b',
                        '6c6462fe-1635-420e-9607-7efa67f75c7e',
                        'e6b40ae9-f63e-4072-9aff-fb33c4538810',
                        '46d50c09-9a8d-4132-815e-b3ae69483b5c',
                        'e6b40ae9-f63e-4072-9aff-fb33c4538810',
                        '46d50c09-9a8d-4132-815e-b3ae69483b5c',
                        'b8a068b8-3e15-4bb4-96d0-ba626ab2cf52',
                        '7532d0a4-37a1-48b9-a572-2304d3cc845e',]

space_instance_parameter_ids = ['cb301144-519f-48dd-9ec7-446b6d7fb4f6',
                                '3a1ac7d2-791d-4e10-b3f3-0146ce97d581',
                                '2770a38e-42d0-423e-9856-d6f452a9f965',
                                '2a5a37ae-45c2-45d9-bdb5-59eba21ec340',
                                '6f2feb4e-aff7-482f-a74b-77c88f22a4b7',
                                'bd0d6db5-b9fe-47d6-8919-2504483a2f9a',
                                'db6608ec-061b-4850-ada8-6850bab94301',
                                '052c3e65-828a-442a-be84-63b504894f12',]

In [51]:
building_id = buildings_dropdown.value

# get site from chosen building
site_id = [x["parent"] for x in project_buildings if x["id"] == building_id][0]

system_type_id = '577cfd8d-8da0-4d78-b4a4-c81ab728d4bf'
sub_system_type_id = "e4b51a06-4f87-4108-9fac-fe44182621fb"

# get all systems and sub systems on project
project_sys = pyddb.get_assets(project_id = project_id,
                              asset_type_id = [system_type_id,
                                              sub_system_type_id])

project_systems = [x for x in project_sys if x["asset_type"]["id"] == system_type_id]
project_subsystems = [x for x in project_sys if x["asset_type"]["id"] == sub_system_type_id]

# get cooling system
cooling_system_id = [x for x in project_systems if x["parent"] == building_id and x["name"] == "Cooling"][0]["id"]

# get cooling fcu sub system
fcu_system_id = [x for x in project_subsystems if x["parent"] == cooling_system_id and x["name"] == fcu_subsystem_name][0]["id"]

# get parameters on the site, building, and fcu system
global_parameters = pyddb.get_parameters(asset_id=[site_id, building_id, fcu_system_id],
                                        parameter_type_id = global_parameter_ids)

space_instance_type_id = '21ba5fbb-f079-4c37-a3b9-f36e21e5b3ac'
space_instance_parameters = pyddb.get_parameters(asset_type = space_instance_type_id,
                                                parameter_type_id = space_instance_parameter_ids,
                                                 project_id = project_id
                                                )

### Space Instance Data Manipulation

Create dictionary of all space instances, their ids, and relevant parameters.

Not currently filtered to those under the selected building. BLOCKED by asset hierarchy in api response/query.

In [52]:
data = [
    (x["parents"][0]["name"],
     x["parameter_type"]["name"],
     x["revision"]["values"][0]["value"])
    for x in space_instance_parameters if x["revision"]]

spaces = list(set([x[0] for x in data]))

space_dict = {
    space: {
        x[1] : x[2]
        for x in data
        if x[0] == space
    }
    for space in spaces}

for space in spaces:
    for x in space_instance_parameters:
        if x["parents"][0]["name"] == space:
            space_dict[space]["id"] = x["parents"][0]["id"]

### Generate DataFrame

In [53]:
input_df = pd.DataFrame.from_dict(space_dict, orient = "index")

In [55]:
new_columns = {"Fresh air flow rate": "Fresh Air Volume Flow Rate",
"Summer internal temperature": "Room Summer Setpoint",
"Sensible heat gain _ winter": "Winter Sensible Heat Gain",
"Sensible heat gain _ summer": "Sensible Heat Gain",
"Latent heat gain _ winter": "Winter Latent Heat Gain",
"Latent heat gain _ summer": "Latent Heat Gain",
"Winter internal temperature": "Room Winter Setpoint",
"Supply air flow rate": "Supply Air Flow Rate"}

In [56]:
input_df=input_df.rename(columns=new_columns)

### Global Cooling Data Manipulation

In [58]:
global_data = [
    (x["parents"][0]["name"],
     x["parameter_type"]["name"],
     x["revision"]["values"][0]["value"])
    for x in global_parameters if x["revision"]]

global_dict = {
        x[1] : x[2]
        for x in global_data}

In [59]:
for k, v in global_dict.items():
    input_df[k] = v

#input_df.index.name = "FCU ID"
input_df["FCU ID"] = input_df.index

### Rename Columns

In [60]:
new_columns = {'Temperature _ return': 'CHW Return Temp',
 'Temperature _ flow': 'CHW Flow Temp',
 'Supply fan temperature rise': 'Supply Fan Temperature Rise',
 'Peak external winter dry bulb temperature': 'Winter Fresh Air Temperature',
 'Peak external summer dry bulb temperature': 'Summer Fresh Air Temperature',
 'Glycol type': 'CHW Glycol Type',
 'Glycol percentage': 'CHW Glycol Concentration',
 'External winter relative humidity': 'Winter Fresh Air Relative Humidity',
 'External summer relative humidity': 'Summer Fresh Air Relative Humidity',
 'Minimum cooling coil temperature': 'Minimum Temperature of Cooling Coil',
 'Cooling coil contact factor': 'Cooling Coil Contact Factor',
 'Altitude': 'Altitude'}

input_df = input_df.rename(columns = new_columns)

### LTHW Flow Temps

In [61]:
# get cooling system
heating_system_id = [x for x in project_systems if x["parent"] == building_id and x["name"] == "Heating"][0]["id"]

# get cooling fcu sub system
fcu_heating_system_id = [x for x in project_subsystems if x["parent"] == heating_system_id and x["name"] == fcu_subsystem_name][0]["id"]


In [62]:
lthw_parameter_ids = ["46d50c09-9a8d-4132-815e-b3ae69483b5c", "e6b40ae9-f63e-4072-9aff-fb33c4538810"]

In [63]:
# get parameters on the site, building, and fcu system
lthw_parameters = pyddb.get_parameters(asset_id = fcu_heating_system_id,
                                        parameter_type_id = lthw_parameter_ids)

In [64]:
lthw_data = [
    (x["parents"][0]["name"],
     x["parameter_type"]["name"],
     x["revision"]["values"][0]["value"])
    for x in lthw_parameters if x["revision"]]

lthw_dict = {
        x[1] : x[2]
        for x in lthw_data}

for k,v in lthw_dict.items():
    input_df[k] = v

In [65]:
new_columns = {'Temperature _ return': 'LTHW Return Temp',
               'Temperature _ flow': 'LTHW Flow Temp'}

input_df = input_df.rename(columns = new_columns)

In [66]:
input_df = input_df.dropna()

In [67]:
output_df_summer, output_df_winter = FCU_batchcalc(job_number, input_df)

In [68]:
input_df

,Room Winter Setpoint,Supply Air Flow Rate,Winter Sensible Heat Gain,Sensible Heat Gain,Winter Latent Heat Gain,Latent Heat Gain,Fresh Air Volume Flow Rate,id,Room Summer Setpoint,CHW Return Temp,...,Minimum Temperature of Cooling Coil,CHW Glycol Type,CHW Glycol Concentration,Winter Fresh Air Relative Humidity,Summer Fresh Air Relative Humidity,Cooling Coil Contact Factor,Altitude,FCU ID,LTHW Return Temp,LTHW Flow Temp
HR 103,19.0,23,-2,0.393,4,4,23,68a8c686-7d0c-41cc-86c6-65e520e3e44d,23.0,13,...,8.5,Ethylene,50,90,42,0.85,5,HR 103,60,80
HR 101,19.0,23,-2,0.343,4,4,23,022aca60-aaa7-4721-afdb-d57b430e255b,23.0,13,...,8.5,Ethylene,50,90,42,0.85,5,HR 101,60,80


In [69]:
output_df_summer

,FCU ID,Supply Air Temperature,Supply Air Relative Humidity,Room Relative Humidity,Cooling Coil Duty,Heating Coil Duty,Rate of Condensate Production,LTHW Mass Flow Rate,CHW Mass Flow Rate
HR 103,HR 103,22.986040,63.500355,63.777002,205.856310,0.0,0.0,0.0,10.620128
HR 101,HR 101,22.987816,63.493530,63.777002,205.805977,0.0,0.0,0.0,10.617531


In [70]:
output_df_winter

,FCU ID,Supply Air Temperature,Supply Air Relative Humidity,Room Relative Humidity,Cooling Coil Duty,Heating Coil Duty,Rate of Condensate Production,LTHW Mass Flow Rate,CHW Mass Flow Rate
HR 103,HR 103,19.071043,30.907241,31.47461,0.0,442.716519,0.0,5.272946,0.0
HR 101,HR 101,19.071043,30.907241,31.47461,0.0,442.716519,0.0,5.272946,0.0


### Post Cooling FCU Product Types

In [71]:
fcu_cooling = output_df_summer.to_dict()
keys = list(fcu_cooling.keys())

In [72]:
fcus = {}

input_dict = input_df.to_dict()

spaces = list(input_dict["FCU ID"].values())

for space in spaces:
    fcus[space] = {}
    for key in keys:
        fcus[space][key] = fcu_cooling[key][space]
        
for space in spaces:
    fcus[space]["id"] = ""
    fcus[space]["id"] = input_dict["id"][space]

In [73]:
output_param_map = {
    'Supply Air Temperature': {
        "id" :'523f8791-2f5f-4358-9029-17538f39f2af',
        "unit":"b39e5bcf-cda6-49ed-8c0c-174d8daf8bc1"
    },
    'Supply Air Relative Humidity':{
        "id" :'1ed136a3-d826-4671-903d-cd88f7085e70',
                "unit":"03252527-2667-4a4a-8f2f-c124233b6847"
    },
    'Cooling Coil Duty':{
        "id" : '6f3a30c8-c9dc-46e6-b1f6-d0872188010a',   
                "unit":"2a029d23-0fcf-4170-93f0-285960937099"
    },
    'Heating Coil Duty':{
        "id" : '9b50dfa1-2ef7-4757-b03e-7adfe5b4afab', 
                "unit":"2a029d23-0fcf-4170-93f0-285960937099"
    },
    'Rate of Condensate Production':{
        "id" :'0f8a1cae-48a8-4449-bcc3-2d58c024f644',       
                "unit":"1295cf94-7c14-4f1e-9126-135040b22ff5"
    },
}

parameter_types = []
values = []
units = []

for k, v in fcus[spaces[0]].items():
    try:
        parameter_types.append(output_param_map[k]["id"])
        values.append(v)
        units.append(output_param_map[k]["unit"])
    except KeyError:
        pass

In [74]:
if parameter_types:
    
    source_id = pyddb.add_new_source(
        project_id = project_id,
        source_type_id = "3169ba8f-d474-4ee3-8891-09817a03b196",
        source_title = "DesignCheck2",
        source_reference = "3603604"
    )
    
    for space in spaces:
        
        asset_id = pyddb.add_new_asset(asset_type_id = '49202b68-1ff9-489b-a8ff-9e7d890f1bf4',  # product on dev
                            project_id = project_id,
                            asset_name = f"FCU - {space}",
                            parent_id = fcu_system_id)
        
        for i, _ in enumerate(parameter_types):

            pyddb.add_update_parameter(
                parameter_type_id = parameter_types[i],
                project_id = project_id,
                asset_id = asset_id,
                value = round(values[i], 2),
                unit_id = units[i],
                source_id = source_id)

Source created successfully.
Asset: FCU - HR 103 created successfully.
Parameter posted successfully.
Parameter posted successfully.
Parameter posted successfully.
Parameter posted successfully.
Parameter posted successfully.
Asset: FCU - HR 101 created successfully.
Parameter posted successfully.
Parameter posted successfully.
Parameter posted successfully.
Parameter posted successfully.
Parameter posted successfully.


### Post Heating FCU Product Types

In [75]:
fcu_heating = output_df_winter.to_dict()
keys = list(fcu_heating.keys())

In [76]:
fcus = {}

input_dict = input_df.to_dict()

spaces = list(input_dict["FCU ID"].values())

for space in spaces:
    fcus[space] = {}
    for key in keys:
        fcus[space][key] = fcu_heating[key][space]
        
for space in spaces:
    fcus[space]["id"] = ""
    fcus[space]["id"] = input_dict["id"][space]

### Swap these out for winter parameter types

In [77]:
output_param_map = {
    'Supply Air Temperature': {
        "id" :'523f8791-2f5f-4358-9029-17538f39f2af',
        "unit":"b39e5bcf-cda6-49ed-8c0c-174d8daf8bc1"
    },
    'Supply Air Relative Humidity':{
        "id" :'1ed136a3-d826-4671-903d-cd88f7085e70',
                "unit":"03252527-2667-4a4a-8f2f-c124233b6847"
    },
    'Cooling Coil Duty':{
        "id" : '6f3a30c8-c9dc-46e6-b1f6-d0872188010a',   
                "unit":"2a029d23-0fcf-4170-93f0-285960937099"
    },
    'Heating Coil Duty':{
        "id" : '9b50dfa1-2ef7-4757-b03e-7adfe5b4afab', 
                "unit":"2a029d23-0fcf-4170-93f0-285960937099"
    },
    'Rate of Condensate Production':{
        "id" :'0f8a1cae-48a8-4449-bcc3-2d58c024f644',       
                "unit":"1295cf94-7c14-4f1e-9126-135040b22ff5"
    },
}

parameter_types = []
values = []
units = []

for k, v in fcus[spaces[0]].items():
    try:
        parameter_types.append(output_param_map[k]["id"])
        values.append(v)
        units.append(output_param_map[k]["unit"])
    except KeyError:
        pass

In [78]:
if parameter_types:
    
    source_id = pyddb.add_new_source(
        project_id = project_id,
        source_type_id = "3169ba8f-d474-4ee3-8891-09817a03b196",
        source_title = "DesignCheck2",
        source_reference = "3603604"
    )
    
    for space in spaces:
        
        asset_id = pyddb.add_new_asset(asset_type_id = '49202b68-1ff9-489b-a8ff-9e7d890f1bf4',  # product on dev
                            project_id = project_id,
                            asset_name = f"FCU - {space}",
                            parent_id = fcu_heating_system_id)
        
        for i, _ in enumerate(parameter_types):

            pyddb.add_update_parameter(
                parameter_type_id = parameter_types[i],
                project_id = project_id,
                asset_id = asset_id,
                value = round(values[i], 2),
                unit_id = units[i],
                source_id = source_id)

Asset: FCU - HR 103 created successfully.
Parameter posted successfully.
Parameter posted successfully.
Parameter posted successfully.
Parameter posted successfully.
Parameter posted successfully.
Asset: FCU - HR 101 created successfully.
Parameter posted successfully.
Parameter posted successfully.
Parameter posted successfully.
Parameter posted successfully.
Parameter posted successfully.
